In [11]:
# Load libraries
import numpy as np
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.losses import MeanAbsoluteError
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.optimizers import Adam

In [12]:
data = pd.read_csv('MF_data.csv')
data

FileNotFoundError: [Errno 2] File b'MF_data.csv' does not exist: b'MF_data.csv'

In [3]:
prices = data['Price'] 
data = data.drop(['Company', 'Time', 'Price'], axis = 1)
data

,Total Revenues,Operating Income,Net Income,Revenue Per Share,Basic EPS,Normalized Basic EPS,Dividend Per Share,EBITA,EBIT,Normalized Net Income
0,6892.3,1147.9,885.0,64.97,8.25,6.17,2.05,1284.4,1147.9,654.6
1,19377.3,2029.1,1281.0,32.81,2.17,2.02,0.69,2029.1,2029.1,1195.1
2,3857.8,583.3,363.5,14.10,1.33,1.12,0.37,676.6,583.3,306.2
3,7676.5,980.2,602.7,22.59,1.76,1.82,0.36,994.0,980.2,619.1
4,155900.0,2658.0,47.0,39.25,0.01,0.40,0.60,2658.0,2658.0,1600.5
5,13378.0,2081.0,961.0,28.48,2.05,1.98,1.36,2132.0,2081.0,929.5
6,7794.0,1756.0,914.0,1.73,0.20,0.19,0.05,1897.0,1756.0,851.9
7,6654.5,2618.9,728.5,8.61,0.94,1.88,0.59,2624.8,2618.9,1449.7
8,10584.0,1510.0,602.0,19.49,1.11,1.59,0.61,1664.0,1510.0,865.6
9,10263.0,1885.0,1439.0,16.88,2.37,1.98,0.00,1921.0,1885.0,1201.3


In [4]:
col_list = []
for col in data.columns:
    col_list.append(col)
    
for col in col_list:
    print(col)
    data[col] = data[col].astype(float) # Converting columns to floats
    print("Done")

# Normalising the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(data) 
data.loc[:,:] = scaled_values

data.insert(data.shape[1], 'Price', prices)
data

Total Revenues
Done
Operating Income
Done
Net Income
Done
Revenue Per Share
Done
Basic EPS
Done
Normalized Basic EPS
Done
Dividend Per Share
Done
EBITA
Done
EBIT
Done
Normalized Net Income
Done


,Total Revenues,Operating Income,Net Income,Revenue Per Share,Basic EPS,Normalized Basic EPS,Dividend Per Share,EBITA,EBIT,Normalized Net Income,Price
0,0.013132,0.152721,0.191876,0.000414,0.000911,0.000826,0.155303,0.151308,0.152721,0.151562,221.83
1,0.036961,0.159877,0.195145,0.000208,0.000790,0.000720,0.052273,0.157298,0.159877,0.158824,26.99
2,0.007341,0.148136,0.187570,0.000089,0.000773,0.000697,0.028030,0.146419,0.148136,0.146882,7.83
3,0.014629,0.151359,0.189545,0.000143,0.000781,0.000715,0.027273,0.148972,0.151359,0.151086,18.38
4,0.297524,0.164984,0.184957,0.000249,0.000746,0.000678,0.045455,0.162357,0.164984,0.164270,9.30
5,0.025511,0.160299,0.192503,0.000181,0.000787,0.000719,0.103030,0.158126,0.160299,0.155255,32.71
6,0.014853,0.157659,0.192115,0.000010,0.000750,0.000673,0.003788,0.156236,0.157659,0.154213,7.15
7,0.012679,0.164667,0.190584,0.000054,0.000765,0.000716,0.044697,0.162090,0.164667,0.162244,46.81
8,0.020178,0.155661,0.189539,0.000123,0.000768,0.000709,0.046212,0.154362,0.155661,0.154397,64.64
9,0.019566,0.158707,0.196450,0.000107,0.000794,0.000719,0.000000,0.156429,0.158707,0.158907,52.66


In [5]:
# Converting Data to Numpy Arrays
NpMatrix = data.to_numpy(dtype = None, copy = False)
X = NpMatrix[:,0:10] # Parameters
y = NpMatrix[:,10] # Price
print("X:", X)
print("\n")
print("Type X:", type(X))
print("Length of Individual X_train Vector:", len(X[1]))
print("Total Number of Training instances:", len(X))
print("\n")
print(y)
print("\n")
print("Type y:", type(y))
print("Length of Individual y_train vector", y[0])
print("Total number of y values", len(y))
number_of_features = len(X[1])

X: [[0.01313249 0.15272091 0.19187598 ... 0.15130826 0.15272091 0.15156246]
 [0.03696098 0.15987705 0.19514547 ... 0.15729821 0.15987705 0.1588236 ]
 [0.00734093 0.14813585 0.18757033 ... 0.14641947 0.14813585 0.14688201]
 ...
 [0.31257509 0.34565004 0.29170268 ... 0.38301227 0.34565004 0.30593264]
 [0.00518902 0.15225639 0.19026601 ... 0.14998432 0.15225639 0.15134751]
 [0.00816982 0.15014171 0.18915389 ... 0.147815   0.15014171 0.14926791]]


Type X: <class 'numpy.ndarray'>
Length of Individual X_train Vector: 10
Total Number of Training instances: 5000


[221.83  26.99   7.83 ...  42.53  50.32  32.35]


Type y: <class 'numpy.ndarray'>
Length of Individual y_train vector 221.83
Total number of y values 5000


In [6]:
#Loss functions
# https://keras.io/api/losses/regression_losses/#meanabsoluteerror-class
# https://keras.io/api/losses/

# Create function returning a compiled network
def create_network():
    
    # Start neural network
    network = models.Sequential()

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=10, activation='relu', input_shape=(number_of_features,)))
    
    #Add Dropout Layer
    network.add(layers.Dropout(0.3))
    
    network.add(layers.Dense(units=5, activation='relu', input_shape=(number_of_features,)))
    
    #Add Dropout Layer
    network.add(layers.Dropout(0.3))

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=1, activation='relu'))

    # Compile neural network
    network.compile(#loss='mse', # Mean Squared Error
                    loss = MeanAbsoluteError(),
                    optimizer='adam', # Root Mean Square Propagation
                    metrics=['mae', 'mse']) # Accuracy performance metric
    
    # Return compiled network
    return network

In [7]:
neural_network = KerasRegressor(build_fn=create_network, 
                                 epochs= 1000, 
                                 batch_size= 5)

In [9]:
# Evaluate neural network using three-fold cross-validation
scores = cross_val_score(neural_network, X, y, cv=5)

Train on 4000 samples
Epoch 1/1000
4000/4000 [==============================] - 3s 627us/sample - loss: 63.4512 - mae: 63.4512 - mse: 24261.8672
Epoch 2/1000
4000/4000 [==============================] - 2s 572us/sample - loss: 52.5306 - mae: 52.5306 - mse: 21873.4902
Epoch 3/1000
4000/4000 [==============================] - 2s 483us/sample - loss: 51.8043 - mae: 51.8043 - mse: 21856.3477
Epoch 4/1000
4000/4000 [==============================] - 2s 513us/sample - loss: 51.7634 - mae: 51.7634 - mse: 21830.6055
Epoch 5/1000
4000/4000 [==============================] - 2s 551us/sample - loss: 51.6705 - mae: 51.6705 - mse: 21745.4902
Epoch 6/1000
4000/4000 [==============================] - 2s 399us/sample - loss: 50.9784 - mae: 50.9784 - mse: 21801.5918
Epoch 7/1000
4000/4000 [==============================] - 2s 527us/sample - loss: 51.1563 - mae: 51.1563 - mse: 21818.8418
Epoch 8/1000
4000/4000 [==============================] - 2s 471us/sample - loss: 50.9142 - mae: 50.9142 - mse: 21741

Epoch 67/1000
4000/4000 [==============================] - 2s 391us/sample - loss: 46.1350 - mae: 46.1350 - mse: 21347.6543
Epoch 68/1000
4000/4000 [==============================] - 2s 382us/sample - loss: 45.9631 - mae: 45.9630 - mse: 21260.8418
Epoch 69/1000
4000/4000 [==============================] - 1s 374us/sample - loss: 46.3271 - mae: 46.3271 - mse: 21340.0332
Epoch 70/1000
4000/4000 [==============================] - 2s 424us/sample - loss: 46.1053 - mae: 46.1054 - mse: 21370.0645
Epoch 71/1000
4000/4000 [==============================] - 2s 378us/sample - loss: 45.7523 - mae: 45.7523 - mse: 21232.2500
Epoch 72/1000
4000/4000 [==============================] - 1s 365us/sample - loss: 45.9178 - mae: 45.9178 - mse: 21274.3828
Epoch 73/1000
4000/4000 [==============================] - 2s 375us/sample - loss: 46.3355 - mae: 46.3355 - mse: 21394.0625
Epoch 74/1000
4000/4000 [==============================] - 2s 423us/sample - loss: 46.2661 - mae: 46.2661 - mse: 21468.1172
Epoch 75

4000/4000 [==============================] - 2s 379us/sample - loss: 44.6634 - mae: 44.6634 - mse: 21133.4238
Epoch 198/1000
4000/4000 [==============================] - 2s 385us/sample - loss: 44.6111 - mae: 44.6111 - mse: 21045.2637
Epoch 199/1000
4000/4000 [==============================] - 2s 420us/sample - loss: 44.6342 - mae: 44.6342 - mse: 21141.4375
Epoch 200/1000
4000/4000 [==============================] - 1s 375us/sample - loss: 45.0685 - mae: 45.0686 - mse: 21175.6035
Epoch 201/1000
4000/4000 [==============================] - 1s 368us/sample - loss: 44.7028 - mae: 44.7028 - mse: 21114.1016
Epoch 202/1000
4000/4000 [==============================] - 2s 427us/sample - loss: 44.7438 - mae: 44.7439 - mse: 21150.0039
Epoch 203/1000
4000/4000 [==============================] - 2s 387us/sample - loss: 44.5170 - mae: 44.5170 - mse: 21073.3652
Epoch 204/1000
4000/4000 [==============================] - 2s 398us/sample - loss: 44.5247 - mae: 44.5247 - mse: 21071.5371
Epoch 205/1000


4000/4000 [==============================] - 2s 418us/sample - loss: 44.4166 - mae: 44.4167 - mse: 20967.1035
Epoch 328/1000
4000/4000 [==============================] - 2s 381us/sample - loss: 44.1909 - mae: 44.1909 - mse: 20893.8926
Epoch 329/1000
4000/4000 [==============================] - 2s 379us/sample - loss: 44.3860 - mae: 44.3860 - mse: 20965.3242
Epoch 330/1000
4000/4000 [==============================] - 2s 402us/sample - loss: 44.8523 - mae: 44.8522 - mse: 21141.4531
Epoch 331/1000
4000/4000 [==============================] - 2s 414us/sample - loss: 44.4867 - mae: 44.4867 - mse: 20977.7031
Epoch 332/1000
4000/4000 [==============================] - 2s 386us/sample - loss: 44.5021 - mae: 44.5021 - mse: 20966.3086
Epoch 333/1000
4000/4000 [==============================] - 2s 393us/sample - loss: 44.1171 - mae: 44.1171 - mse: 20904.6895
Epoch 334/1000
4000/4000 [==============================] - 2s 416us/sample - loss: 44.8572 - mae: 44.8572 - mse: 21048.4629
Epoch 335/1000


4000/4000 [==============================] - 2s 392us/sample - loss: 44.3793 - mae: 44.3793 - mse: 20964.6309
Epoch 458/1000
4000/4000 [==============================] - 2s 398us/sample - loss: 44.5161 - mae: 44.5161 - mse: 20971.7363
Epoch 459/1000
4000/4000 [==============================] - 1s 374us/sample - loss: 44.4052 - mae: 44.4052 - mse: 20977.7637
Epoch 460/1000
4000/4000 [==============================] - 2s 421us/sample - loss: 44.3196 - mae: 44.3196 - mse: 20956.3809
Epoch 461/1000
4000/4000 [==============================] - 2s 392us/sample - loss: 44.4065 - mae: 44.4065 - mse: 20996.7676
Epoch 462/1000
4000/4000 [==============================] - 2s 387us/sample - loss: 44.5863 - mae: 44.5863 - mse: 21045.5000
Epoch 463/1000
4000/4000 [==============================] - 2s 413us/sample - loss: 44.5927 - mae: 44.5926 - mse: 21000.9434
Epoch 464/1000
4000/4000 [==============================] - 2s 394us/sample - loss: 44.2075 - mae: 44.2075 - mse: 20957.1328
Epoch 465/1000


...

4000/4000 [==============================] - 3s 634us/sample - loss: 43.8853 - mae: 43.8853 - mse: 20756.1465
Epoch 458/1000
4000/4000 [==============================] - 3s 704us/sample - loss: 43.9010 - mae: 43.9010 - mse: 20748.3906
Epoch 459/1000
4000/4000 [==============================] - 3s 686us/sample - loss: 44.0413 - mae: 44.0412 - mse: 20768.1270
Epoch 460/1000
4000/4000 [==============================] - 3s 724us/sample - loss: 43.6360 - mae: 43.6360 - mse: 20673.8340
Epoch 461/1000
4000/4000 [==============================] - 2s 608us/sample - loss: 43.7898 - mae: 43.7898 - mse: 20713.6191
Epoch 462/1000
4000/4000 [==============================] - 3s 689us/sample - loss: 43.7643 - mae: 43.7643 - mse: 20695.6172
Epoch 463/1000
4000/4000 [==============================] - 2s 604us/sample - loss: 43.6419 - mae: 43.6419 - mse: 20713.2715
Epoch 464/1000
4000/4000 [==============================] - 3s 700us/sample - loss: 43.9139 - mae: 43.9139 - mse: 20746.8203
Epoch 465/1000


4000/4000 [==============================] - 2s 599us/sample - loss: 43.6776 - mae: 43.6776 - mse: 20644.1973
Epoch 588/1000
4000/4000 [==============================] - 2s 593us/sample - loss: 43.1694 - mae: 43.1694 - mse: 20608.1699
Epoch 589/1000
4000/4000 [==============================] - 2s 614us/sample - loss: 43.6957 - mae: 43.6957 - mse: 20703.0469
Epoch 590/1000
4000/4000 [==============================] - 2s 489us/sample - loss: 43.4683 - mae: 43.4683 - mse: 20631.7578
Epoch 591/1000
4000/4000 [==============================] - 2s 408us/sample - loss: 43.8681 - mae: 43.8681 - mse: 20679.2227
Epoch 592/1000
4000/4000 [==============================] - 2s 419us/sample - loss: 43.7596 - mae: 43.7596 - mse: 20722.9258
Epoch 593/1000
4000/4000 [==============================] - 2s 381us/sample - loss: 44.1984 - mae: 44.1984 - mse: 20813.9512
Epoch 594/1000
4000/4000 [==============================] - 2s 397us/sample - loss: 43.7462 - mae: 43.7462 - mse: 20698.1367
Epoch 595/1000


4000/4000 [==============================] - 3s 654us/sample - loss: 43.7526 - mae: 43.7526 - mse: 20669.3711
Epoch 718/1000
4000/4000 [==============================] - 3s 692us/sample - loss: 43.6743 - mae: 43.6744 - mse: 20674.8711
Epoch 719/1000
4000/4000 [==============================] - 3s 723us/sample - loss: 43.9510 - mae: 43.9510 - mse: 20750.6797
Epoch 720/1000
4000/4000 [==============================] - 2s 392us/sample - loss: 43.8552 - mae: 43.8552 - mse: 20812.0645
Epoch 721/1000
4000/4000 [==============================] - 2s 395us/sample - loss: 44.0137 - mae: 44.0137 - mse: 20787.5820
Epoch 722/1000
4000/4000 [==============================] - 2s 388us/sample - loss: 43.5134 - mae: 43.5134 - mse: 20618.0938
Epoch 723/1000
4000/4000 [==============================] - 2s 422us/sample - loss: 43.6839 - mae: 43.6839 - mse: 20712.1133
Epoch 724/1000
4000/4000 [==============================] - 2s 383us/sample - loss: 43.5309 - mae: 43.5309 - mse: 20595.9961
Epoch 725/1000


4000/4000 [==============================] - 2s 537us/sample - loss: 44.0184 - mae: 44.0184 - mse: 20767.0977
Epoch 848/1000
4000/4000 [==============================] - 2s 411us/sample - loss: 43.6081 - mae: 43.6081 - mse: 20659.2188
Epoch 849/1000
4000/4000 [==============================] - 2s 382us/sample - loss: 43.9703 - mae: 43.9703 - mse: 20724.1562
Epoch 850/1000
4000/4000 [==============================] - 2s 411us/sample - loss: 43.7346 - mae: 43.7346 - mse: 20710.2969
Epoch 851/1000
4000/4000 [==============================] - 2s 389us/sample - loss: 44.0060 - mae: 44.0061 - mse: 20793.3262
Epoch 852/1000
4000/4000 [==============================] - 2s 420us/sample - loss: 43.7795 - mae: 43.7794 - mse: 20694.8789
Epoch 853/1000
4000/4000 [==============================] - 2s 395us/sample - loss: 43.7758 - mae: 43.7758 - mse: 20687.6973
Epoch 854/1000
4000/4000 [==============================] - 2s 377us/sample - loss: 43.9765 - mae: 43.9765 - mse: 20726.5781
Epoch 855/1000


4000/4000 [==============================] - 2s 381us/sample - loss: 43.3909 - mae: 43.3909 - mse: 20575.7773
Epoch 978/1000
4000/4000 [==============================] - 2s 419us/sample - loss: 43.7774 - mae: 43.7774 - mse: 20716.2773
Epoch 979/1000
4000/4000 [==============================] - 2s 388us/sample - loss: 43.7203 - mae: 43.7203 - mse: 20750.7734
Epoch 980/1000
4000/4000 [==============================] - 2s 395us/sample - loss: 43.7155 - mae: 43.7155 - mse: 20718.4141
Epoch 981/1000
4000/4000 [==============================] - 2s 477us/sample - loss: 43.8559 - mae: 43.8559 - mse: 20720.1914
Epoch 982/1000
4000/4000 [==============================] - 2s 429us/sample - loss: 43.8117 - mae: 43.8116 - mse: 20703.6426
Epoch 983/1000
4000/4000 [==============================] - 2s 420us/sample - loss: 43.6538 - mae: 43.6538 - mse: 20679.9551
Epoch 984/1000
4000/4000 [==============================] - 2s 403us/sample - loss: 43.6890 - mae: 43.6890 - mse: 20744.5312
Epoch 985/1000


In [10]:
print(scores)
print("Mean MAE:", scores.mean())
print("Standard Deviation of the mean:", scores.std())

[-37.68065058 -87.79746021 -37.18356174 -40.29338048 -40.46431961]
Mean MAE: -48.683874525547026
Standard Deviation of the mean: 19.601847820388226
